# GitHub Integration & Automation Tutorial

This tutorial demonstrates how to build complete GitHub automation workflows, starting with manual git operations and progressing to advanced API integration using Composio.

## What You'll Learn

- **Part 1**: Manual Git Operations - Automate git commands using Python subprocess
- **Part 2**: Advanced GitHub Integration - Use Composio for GitHub API operations
- **Part 3**: Complete Automation Workflow - Combine both approaches for production use

## Prerequisites

- Python 3.8+
- Git installed on your system
- GitHub account
- For advanced features: Composio API key and OpenRouter API key

## Tutorial Architecture

```
Manual Git Operations → GitHub API Integration → Complete Automation
     (subprocess)           (Composio)              (Combined)
```


## Part 1: Manual Git Operations with Python

Let's start by automating basic git operations using Python's subprocess module. This approach gives you full control and doesn't require external API keys.


### 1.1 Package Installation and Setup

First, let's install the required packages and set up our workspace.


In [14]:
# !pip install composio langchain langchain-openai langchain-core openai python-dotenv discord.py langsmith langgraph

In [15]:
# Install required packages

import os
import time
from typing import Optional, Dict
import subprocess
import sys
import shutil

# Environment setup for Composio
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [ ]:
# Configuration for this tutorial

# Select the workspace for the tutorial
# GitHub Repositories will be cloned in this directory

TUTORIAL_WORKSPACE = "/Users/ishandutta/Documents/code/orion-accelerator"
os.makedirs(TUTORIAL_WORKSPACE, exist_ok=True)
print(f"📁 Tutorial workspace created: {TUTORIAL_WORKSPACE}")

📁 Tutorial workspace created: /Users/ishandutta/Documents/code/orion-accelerator


### 1.2 Complete Git Workflow Functions

Let's create the complete set of git operation functions based on the working implementation.


In [17]:
def clone_or_update_repository(repo_url: str, repo_path: str) -> bool:
    """
    Clone GitHub repository to desired location.

    Args:
        repo_url: GitHub repository URL
        repo_path: Path where to clone the repository

    Returns:
        bool: True if successful, False otherwise
    """
    print(f"🔄 Cloning repository...")

    subprocess.run(
        ["git", "clone", repo_url, repo_path], check=True, capture_output=True
    )

    print("✅ Repository ready")
    return True

In [18]:
def clone_or_update_repository(repo_url: str, repo_path: str) -> bool:
    """
    Clone repository or update if it exists.

    Args:
        repo_url: GitHub repository URL
        repo_path: Path where to clone the repository

    Returns:
        bool: True if successful
    """
    if os.path.exists(repo_path):
        print(f"📁 Repository exists, updating...")
        original_dir = os.getcwd()
        os.chdir(repo_path)

        subprocess.run(["git", "fetch", "origin"], check=True, capture_output=True)
        subprocess.run(["git", "checkout", "main"], check=True, capture_output=True)
        subprocess.run(
            ["git", "pull", "origin", "main"], check=True, capture_output=True
        )

        os.chdir(original_dir)

    else:
        print(f"🔄 Cloning repository...")
        subprocess.run(
            ["git", "clone", repo_url, repo_path], check=True, capture_output=True
        )

    print("✅ Repository ready")
    return True

In [28]:
def create_and_switch_branch(branch_name: str, repo_path: str) -> bool:
    """
    Create and switch to new branch.

    Args:
        branch_name: Name of the branch to create
        repo_path: Path to the git repository

    Returns:
        bool: True if successful, False otherwise
    """
    original_dir = os.getcwd()
    os.chdir(repo_path)

    # Create and switch to new branch
    subprocess.run(
        ["git", "checkout", "-b", branch_name], check=True, capture_output=True
    )
    print(f"🌿 Created and switched to branch: {branch_name}")

    os.chdir(original_dir)
    return True

In [29]:
def make_changes_and_commit(repo_path: str, branch_name: str, description: str) -> bool:
    """
    Make sample changes and commit them.

    Args:
        repo_path: Path to the git repository
        branch_name: Current branch name
        description: Description of changes being made

    Returns:
        bool: True if successful, False otherwise
    """
    # Create a test file
    test_file_path = os.path.join(repo_path, f"orion_tutorial_{int(time.time())}.md")

    test_content = f"""# Orion Tutorial Demo

This file was created automatically by the Orion GitHub integration tutorial.

## Details
- **Timestamp**: {time.strftime('%Y-%m-%d %H:%M:%S')}
- **Branch**: {branch_name}
- **Description**: {description}

## Features Demonstrated
- Automated git operations
- Branch management with unique naming
- File creation and modification
- Commit automation with proper formatting
- Push and PR creation

## Workflow Steps
1. ✅ Repository cloned/updated
2. ✅ Branch created and switched
3. ✅ File created and changes made
4. ✅ Changes committed
5. 🔄 Branch push (next step)
6. 🔄 Pull request creation (final step)

This demonstrates the complete automation capabilities of the Orion system.
"""

    with open(test_file_path, "w") as f:
        f.write(test_content)

    print(f"📝 Created test file: {os.path.basename(test_file_path)}")

    # Commit changes
    original_dir = os.getcwd()
    os.chdir(repo_path)

    subprocess.run(["git", "add", "."], check=True, capture_output=True)
    commit_message = f":robot: [orion] {description}"
    subprocess.run(
        ["git", "commit", "-m", commit_message], check=True, capture_output=True
    )

    print(f"💾 Changes committed with message: {commit_message}")
    os.chdir(original_dir)
    return True

In [30]:
def push_branch(repo_path: str, branch_name: str) -> bool:
    """
    Push branch to remote - REQUIRED before creating PR.

    Args:
        repo_path: Path to the git repository
        branch_name: Name of the branch to push

    Returns:
        bool: True if successful, False otherwise
    """
    original_dir = os.getcwd()
    os.chdir(repo_path)

    subprocess.run(
        ["git", "push", "-u", "origin", branch_name], check=True, capture_output=True
    )
    print(f"🚀 Branch {branch_name} pushed to remote")

    os.chdir(original_dir)
    return True


print("✅ Git workflow functions defined")
print(
    "💡 These functions handle the complete git workflow: clone → branch → commit → push"
)

✅ Git workflow functions defined
💡 These functions handle the complete git workflow: clone → branch → commit → push


### 1.3 Test Basic Git Operations

Let's test the git operations with a real repository.


In [31]:
# Test git operations
test_repo_url = "https://github.com/ishandutta0098/open-clip.git"
test_repo_name = "open-clip"
test_repo_path = os.path.join(TUTORIAL_WORKSPACE, test_repo_name)

# Generate unique branch name
test_branch_name = f"orion/tutorial-git-test-{int(time.time())}"

print(f"🧪 Testing Git Operations")
print(f"📁 Repository: {test_repo_url}")
print(f"🌿 Branch: {test_branch_name}")
print()

# Step 1: Clone or update repository
clone_or_update_repository(test_repo_url, test_repo_path)
print("\n✅ Step 1: Repository ready")

# Step 2: Create and switch to new branch
create_and_switch_branch(test_branch_name, test_repo_path)
print("\n✅ Step 2: Branch created")
print(f"🌿 Branch created: {test_branch_name}")

# Step 3: Make changes and commit
make_changes_and_commit(
    test_repo_path,
    test_branch_name,
    "Add tutorial demo file with git automation",
)

print("\n✅ Step 3: Changes committed")
print("📝 Changes committed and ready for push")

🧪 Testing Git Operations
📁 Repository: https://github.com/ishandutta0098/open-clip.git
🌿 Branch: orion/tutorial-git-test-1758950045

🔄 Cloning repository...
✅ Repository ready

✅ Step 1: Repository ready
🌿 Created and switched to branch: orion/tutorial-git-test-1758950045

✅ Step 2: Branch created
🌿 Branch created: orion/tutorial-git-test-1758950045
📝 Created test file: orion_tutorial_1758950050.md
💾 Changes committed with message: :robot: [orion] Add tutorial demo file with git automation

✅ Step 3: Changes committed
📝 Changes committed and ready for push


## Part 2: Advanced GitHub Integration with Composio

Now let's integrate with GitHub API using Composio for advanced operations like creating pull requests.


### 2.1 Initialize Composio and OpenRouter Clients

Set up authentication with GitHub through Composio and OpenRouter for AI API calls.


In [32]:
from composio import Composio
from openai import OpenAI

# Initialize Composio client
composio_client = Composio(api_key=os.getenv("COMPOSIO_API_KEY"))
user_id = os.getenv("USER_ID")

# Initialize OpenAI client with OpenRouter for cost-effective API access
openai_client = OpenAI(
    base_url="https://openrouter.ai/api/v1", 
    api_key=os.getenv("OPENROUTER_API_KEY")
)

print("🔧 Composio and OpenRouter clients initialized")

# Test authentication
tools = composio_client.tools.get(user_id=user_id, toolkits=["GITHUB"])
print(f"✅ GitHub authentication successful! Found {len(tools)} GitHub tools.")

🔧 Composio and OpenRouter clients initialized
✅ GitHub authentication successful! Found 20 GitHub tools.


### 2.2 Working Pull Request Creation Function

In [33]:
def extract_pr_url(result) -> Optional[str]:
    """
    Extract PR URL from Composio API response - exactly like working agent.
    """
    try:
        if not result or not isinstance(result, list):
            return None

        for item in result:
            if "data" in item:
                pr_data = item["data"]
                if isinstance(pr_data, dict):
                    return pr_data.get("html_url")

        return None

    except Exception as e:
        print(f"Error extracting PR URL: {e}")
        return None

print("✅ GitHub PR creation functions defined")
print("💡 Ready to create pull requests using Composio + OpenRouter")

✅ GitHub PR creation functions defined
💡 Ready to create pull requests using Composio + OpenRouter


In [34]:
def create_github_pull_request(
    repo_url: str, title: str, body: str, branch_name: str
) -> Dict:
    """
    Create a GitHub pull request using Composio.

    Args:
        repo_url: GitHub repository URL
        title: Pull request title
        body: Pull request body/description
        branch_name: Source branch name for the PR (must exist on remote)

    Returns:
        Dict: {"success": bool, "pr_url": str, "result": dict}
    """
    # Get specific GitHub PR creation tools first
    pr_tools = ["GITHUB_CREATE_A_PULL_REQUEST", "GITHUB_PULLS_CREATE"]
    tools = composio_client.tools.get(user_id=user_id, tools=pr_tools)
    print(f"🔍 Available GitHub tools")
    print(tools)

    # Extract owner and repo from URL
    repo_parts = repo_url.replace(".git", "").split("/")
    owner = repo_parts[-2]
    repo = repo_parts[-1]

    print(f"🚀 Creating pull request for {owner}/{repo}...")
    print(f"   📋 Title: {title}")
    print(f"   🌿 Branch: {branch_name} → main")

    # Format title with orion prefix
    if not title.startswith(":robot: [orion]"):
        formatted_title = f":robot: [orion] {title}"
    else:
        formatted_title = title

    # Create task for PR creation
    task = f"""Create a pull request in the GitHub repository {owner}/{repo} with the following details:
    - Title: {formatted_title}
    - Body: {body}
    - Head branch: {branch_name}
    - Base branch: main
    
    Please use the GitHub API to create this pull request and return the PR details including URL, number, and status."""

    print(f"🔍 Available GitHub tools: {len(tools)}")

    # Create chat completion request
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that creates GitHub pull requests using the available GitHub tools.",
            },
            {"role": "user", "content": task},
        ],
        tools=tools,
    )
    print("\nResponse")
    print(response)

    if (
        hasattr(response.choices[0].message, "tool_calls")
        and response.choices[0].message.tool_calls
    ):
        print(
            f"🔍 Tool calls generated: {len(response.choices[0].message.tool_calls)}"
        )
        for i, tool_call in enumerate(response.choices[0].message.tool_calls):
            print(f"🔍 Tool call {i+1}: {tool_call.function.name}")
    else:
        print("⚠️ No tool calls generated by OpenAI")

    # Handle tool calls
    result = composio_client.provider.handle_tool_calls(
        user_id=user_id, response=response
    )
    print("\nResult")
    print(result)

    # Extract PR URL
    pr_url = extract_pr_url(result)

    if pr_url:
        print(f"✅ Pull request created successfully!")
        print(f"🔗 PR URL: {pr_url}")
        return {"success": True, "pr_url": pr_url, "result": result}
    else:
        print("⚠️ Pull request creation may have failed")
        print(f"📝 Raw result: {result}")
        return {"success": False, "pr_url": None, "result": result}

## Part 3: Complete Automation Workflow

Now let's combine everything into a complete workflow: clone → branch → commit → push → PR.


In [36]:
def complete_github_workflow(
    repo_url: str,
    workflow_description: str,
    workspace_path: str = TUTORIAL_WORKSPACE,
    create_pr: bool = True,
) -> Dict:
    """
    Execute complete GitHub automation workflow.

    Args:
        repo_url: GitHub repository URL
        workflow_description: Description of changes being made
        workspace_path: Local workspace path
        create_pr: Whether to create a pull request

    Returns:
        Dict: Workflow results with success status and details
    """
    print("🚀 Starting Complete GitHub Automation Workflow")
    print("=" * 60)

    # Extract repo name and setup paths
    repo_name = repo_url.split("/")[-1].replace(".git", "")
    repo_path = os.path.join(workspace_path, repo_name)

    # Generate unique branch name
    branch_name = f"orion/tutorial-complete-{int(time.time())}"

    print(f"📁 Repository: {repo_url}")
    print(f"🌿 Branch: {branch_name}")
    print(f"📝 Description: {workflow_description}")
    print()

    workflow_result = {
        "success": False,
        "steps_completed": [],
        "repo_path": repo_path,
        "branch_name": branch_name,
        "pr_url": None,
        "errors": [],
    }

    try:
        # Step 1: Clone or update repository
        print("📦 Step 1: Repository Setup")
        if not clone_or_update_repository(repo_url, repo_path):
            workflow_result["errors"].append("Repository setup failed")
            return workflow_result
        workflow_result["steps_completed"].append("repository_setup")

        # Step 2: Create and switch to new branch
        print("\\n🌿 Step 2: Branch Creation")
        if not create_and_switch_branch(branch_name, repo_path):
            workflow_result["errors"].append("Branch creation failed")
            return workflow_result
        workflow_result["steps_completed"].append("branch_creation")

        # Step 3: Make changes and commit
        print("\\n✏️ Step 3: Make Changes and Commit")
        if not make_changes_and_commit(repo_path, branch_name, workflow_description):
            workflow_result["errors"].append("Changes and commit failed")
            return workflow_result
        workflow_result["steps_completed"].append("changes_committed")

        # Step 4: Push branch to remote
        print("\\n🚀 Step 4: Push Branch to Remote")
        if not push_branch(repo_path, branch_name):
            workflow_result["errors"].append("Branch push failed")
            return workflow_result
        workflow_result["steps_completed"].append("branch_pushed")

        # Step 5: Create pull request (if enabled and possible)
        if create_pr:
            print("\\n🔗 Step 5: Create Pull Request")

            pr_title = f"Tutorial: {workflow_description}"
            pr_body = f"""This pull request was created automatically by the Orion GitHub automation tutorial.

## Changes Made
{workflow_description}

## Workflow Steps Completed
1. ✅ Repository cloned/updated
2. ✅ Branch created: `{branch_name}`
3. ✅ Changes made and committed
4. ✅ Branch pushed to remote
5. ✅ Pull request created

This demonstrates the complete GitHub automation capabilities using:
- **Manual Git Operations**: Python subprocess for git commands
- **GitHub API Integration**: Composio for PR creation
- **AI-Powered API Calls**: OpenRouter for cost-effective AI access

**Generated by**: Orion AI Agent System Tutorial
"""

            pr_result = create_github_pull_request(
                repo_url, pr_title, pr_body, branch_name
            )

            if pr_result and pr_result.get("success"):
                workflow_result["pr_url"] = pr_result["pr_url"]
                workflow_result["steps_completed"].append("pull_request_created")
                print(f"✅ Pull request created: {pr_result['pr_url']}")
            else:
                workflow_result["errors"].append(f"PR creation failed: {pr_result}")
                print(
                    "⚠️ Pull request creation failed, but workflow otherwise successful"
                )

        elif create_pr:
            print("\\n💡 Step 5: Pull Request Skipped")
            print("   Reason: Advanced features not available or not authenticated")
            workflow_result["steps_completed"].append("pr_skipped")

        workflow_result["success"] = True
        print("\\n🎉 Workflow completed successfully!")

    except Exception as e:
        error_msg = f"Workflow failed: {str(e)}"
        print(f"\\n❌ {error_msg}")
        workflow_result["errors"].append(error_msg)

    return workflow_result


print("✅ Complete workflow function defined")
print("💡 Ready to execute end-to-end GitHub automation")

✅ Complete workflow function defined
💡 Ready to execute end-to-end GitHub automation


### 3.1 Execute Complete Workflow

Let's run the complete automation workflow to demonstrate all capabilities.


In [37]:
# Execute the complete workflow
workflow_result = complete_github_workflow(
    repo_url="https://github.com/ishandutta0098/open-clip.git",
    workflow_description="Complete automation tutorial demonstration with all features",
    create_pr=True,  # Set to False if you don't want to create a PR
)

# Display comprehensive results
print("\\n📊 Workflow Results Summary")
print("=" * 50)
print(f"✅ Success: {workflow_result['success']}")
print(f"📦 Repository: {workflow_result['repo_path']}")
print(f"🌿 Branch: {workflow_result['branch_name']}")
print(f"📋 Completed Steps: {', '.join(workflow_result['steps_completed'])}")

if workflow_result["errors"]:
    print(f"❌ Errors: {', '.join(workflow_result['errors'])}")

if workflow_result["pr_url"]:
    print(f"🔗 Pull Request: {workflow_result['pr_url']}")

print("\\n🎯 Workflow Demonstrates:")
print("   • Manual git operations with Python subprocess")
print("   • Intelligent branch naming and management")
print("   • Automated file creation and commits")
print("   • GitHub API integration via Composio")
print("   • Cost-effective AI API usage with OpenRouter")
print("   • Complete end-to-end automation")

🚀 Starting Complete GitHub Automation Workflow
📁 Repository: https://github.com/ishandutta0098/open-clip.git
🌿 Branch: orion/tutorial-complete-1758950979
📝 Description: Complete automation tutorial demonstration with all features

📦 Step 1: Repository Setup
📁 Repository exists, updating...
✅ Repository ready
\n🌿 Step 2: Branch Creation
🌿 Created and switched to branch: orion/tutorial-complete-1758950979
\n✏️ Step 3: Make Changes and Commit
📝 Created test file: orion_tutorial_1758950981.md
💾 Changes committed with message: :robot: [orion] Complete automation tutorial demonstration with all features
\n🚀 Step 4: Push Branch to Remote
🚀 Branch orion/tutorial-complete-1758950979 pushed to remote
\n🔗 Step 5: Create Pull Request
🔍 Available GitHub tools
[{'function': {'name': 'GITHUB_CREATE_A_PULL_REQUEST', 'description': 'Creates a pull request in a github repository, requiring existing `base` and `head` branches; `title` or `issue` must be provided.', 'parameters': {'description': 'Request 

## Summary and Key Learnings

### What We've Accomplished

✅ **Complete GitHub Automation Workflow**:
1. **Repository Management**: Clone and update repositories intelligently
2. **Branch Operations**: Create unique branches with collision avoidance
3. **File Modifications**: Automated file creation and content generation
4. **Commit Management**: Proper commit formatting with automation prefixes
5. **Remote Synchronization**: Push branches to remote repositories
6. **Pull Request Creation**: API-driven PR creation using Composio

### Critical Success Factors

🔑 **Key Insight**: **Always push the branch before creating a PR**
- This was the root cause of the "Repository owner cannot be a collaborator" error
- GitHub API requires the branch to exist on remote before PR creation

🔑 **Working Implementation**:
- Use specific PR tools first: `["GITHUB_CREATE_A_PULL_REQUEST", "GITHUB_PULLS_CREATE"]`
- Fallback to general GitHub tools if specific ones aren't available
- Exact same task format and URL extraction as `GitHubIntegrationAgent`

### Technology Stack Benefits

💰 **Cost-Effective**: OpenRouter provides significant savings over OpenAI
- Free tiers available for development
- Multiple model options including open-source
- OpenAI-compatible API format

🔧 **Reliable**: Subprocess-based git operations
- Direct control over git commands
- Comprehensive error handling
- Works with any git repository

🚀 **Scalable**: Composio GitHub integration
- Professional API access management
- Rate limiting and error handling
- Multiple GitHub operations supported

### Production Considerations

When implementing this in production:

- **Security**: Use proper secret management systems
- **Error Handling**: Implement comprehensive retry mechanisms
- **Monitoring**: Add detailed logging and metrics
- **Testing**: Test thoroughly with your specific repositories
- **Permissions**: Ensure proper GitHub repository permissions
- **Rate Limits**: Respect API rate limits and implement backoff

### Next Steps

🎯 **Tutorial 2**: AI-Powered Code Generation
- Learn about intelligent code analysis and generation
- Repository scanning and modification detection
- Context-aware code improvements

🎯 **Tutorial 3**: LangGraph Orchestration
- Advanced workflow orchestration
- Multi-agent coordination
- Complex decision-making workflows

---

**🎉 Congratulations!** You now have a complete, working GitHub automation system that can:
- Clone repositories and manage branches intelligently
- Make automated code changes and commits
- Create pull requests via GitHub API
- Handle errors gracefully and provide detailed feedback

This forms the foundation for building sophisticated AI-powered development workflows!
